<a href="https://colab.research.google.com/github/AlexeyEvzrezov/DS_projects/blob/master/data_conversion_and_matrix_operations/data_conversion_and_matrix_operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Реализация метода преобразования данных для защиты персональной информации

В настоящей работе реализован механизм защититы данных клиентов страховой компании. 
Данные защищены таким образом, что при преобразовании качество моделей машинного обучения не ухудшается. 

## Содержание
- [Описание данных](#0)
1. [Загрузка данных](#1)    
2. [Умножение матриц](#2)    
3. [Алгоритм преобразования](#3)
3. [Проверка алгоритма](#4)
- [Выводы](#5)

<a id="0"></a> 
## Описание данных
- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

<a id="1"></a> 
## Загрузка данных

In [ ]:
# импорт библиотек
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

Загрузим данные и посмотрим на них.

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Datasets/insurance.csv')                 

In [ ]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.000,"49,600.000",1,0
1,0,46.000,"38,000.000",1,1
2,0,29.000,"21,000.000",0,0
3,0,21.000,"41,700.000",2,0
4,1,28.000,"26,100.000",0,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [ ]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,"5,000.000","5,000.000","5,000.000","5,000.000","5,000.000"
mean,0.499,30.953,"39,916.360",1.194,0.148
std,0.500,8.441,"9,900.084",1.091,0.463
min,0.000,18.000,"5,300.000",0.000,0.000
25%,0.000,24.000,"33,300.000",0.000,0.000
50%,0.000,30.000,"40,200.000",1.000,0.000
75%,1.000,37.000,"46,600.000",2.000,0.000
max,1.000,65.000,"79,000.000",6.000,5.000


Данные в порядке и пригодны для нашей работы.

<a id="2"></a> 
## 2. Умножение матриц

Покажем аналитически, что качество линейной регрессии не изменится при умножении признаков на обратимую матрицу.  
В этом случае ответы алгоритма на преобразованных таким образом данных не должны отличаться от ответов на исходных данных.

Введем следующие обозначения:
- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $y$ — вектор целевого признака

- $a$ — вектор предсказаний целевого признака

Уравнение линейной регрессии с учетом введенных обозначений имеет вид:  

$$
a = Xw \;\;\;\;\;\;\;\;\text{(1)}
$$  

При этом вектор весов $w$ определяется выражением:  
$$
w = (X^T X)^{-1} X^T y \;\;\;\;\;\;\;\;\text{(2)}
$$

Выразим $w$ в (1) через (2) и подставим преобразованную матрицу $XP$ вместо $X$:  
  
$$
a = XP((XP)^T XP)^{-1} (XP)^T y
$$ 


Используя свойства транспонирования получим:  

$$
a = XP(P^T X^T XP)^{-1} P^T X^T y
$$ 


Используя свойства обратных матриц получим:  

$$
a = XP(X^T XP)^{-1} (P^T)^{-1}  P^T X^T y
$$ 


$$
a = XP P^{-1}(X^T X)^{-1} (P^T)^{-1}  P^T X^T y
$$ 

Учитывая свойство $A^{-1}A=AA^{-1}=E$, имеем:  

$$
a = XE(X^T X)^{-1} E X^T y
$$ 

$$
a = X(X^T X)^{-1} X^T y
$$

$$
a = Xw,
$$ 
что и требовалось доказать.  

Иными словами, мы можем представить модель в виде:
$$
a = XPP^{-1}w
$$ 
Таким образом, меняются только веса, а предсказания остаются неизменными.

<a id="3"></a> 
## 3. Алгоритм преобразования


Алгоритм преобразования будет данных выглядеть следующим образом:
- Выделение матрицы признаков и вектора целевых переменных;  
- Генерация случайной обратимой матрицы подходящей размерности (квадратной, с количеством столбцов (строк) равным количеству признаков);
- Умножение матрицы признаков на сгенерированную матрицу.

Обоснованием алгоритма являются выводы, полученные нами в предыдущем разделе.

<a id="4"></a> 
## 4. Проверка алгоритма

Инициализируем условия для генератора случайных чисел.

In [ ]:
np.random.seed(0)

Напишем функцию для генерации случайной обратимой квадратной матрицы.

In [ ]:
def rand_inv(m):
    """
    Square invertible matrix with random values.
    Parameters
    ----------
    m : int
        The dimension of the returned matrix
    Returns
    -------
    out: ndarray, shape (m, m)
    """  
    x = np.random.rand(m, m)
    if np.linalg.det(x) == 0: 
        rand_inv(m)
        
    return np.linalg.inv(x)

Выделим матрицу признаков и вектор целевых переменных.

In [ ]:
X, y = df.iloc[:, :-1], df.iloc[:, -1]

Построим модель линейной регрессии на исходных данных и оценим коэффициент детерминации на кросс-валидации.

In [ ]:
lr = LinearRegression()
r2_before = cross_val_score(lr, X, y, scoring='r2')
r2_before

array([0.40104201, 0.44663077, 0.41583058, 0.41470783, 0.43735766])

Преобразуем исходные данные.

In [ ]:
X_new = X.values @ rand_inv(X.shape[1]) 

 На кросс-валидации оценим коэффициент детерминации на преобразованных данных.

In [ ]:
r2_after = cross_val_score(lr, X_new, y, scoring='r2')
r2_after

array([0.40104201, 0.44663077, 0.41583058, 0.41470783, 0.43735766])

Очевидно, что полученные значения не отличаются.

Сгенерируем 100 случайных матриц преобразования и покажем, что качество не изменится.

In [ ]:
for i in range(100):
    key = rand_inv(X.shape[1])
    X_new = X.values @ key
    r2_after = cross_val_score(lr, X_new, y, scoring='r2')
    if str(r2_after) != str(r2_before):   # переводим в строки, чтобы исключить ошибки, связанные со спецификой представления float
        print('Качество модели после преобразования отличается')
    elif i == 99:
        print('Качество модели после преобразования не отличается') 

Качество модели после преобразования не отличается


Теперь выполним обратное преобразование.  
Умножим преобразованную матрицу признаков на инвертированную матрицу преобразования, которая и является ключом.  
Сравним полученные значения, округлив их до трех знаков после запятой.

In [ ]:
(np.around(X_new @ np.linalg.inv(key), 3) != np.around(X, 3)).sum().sum()

0

Таким образом мы видим, что все значения совпали.

<a id="5"></a> 
## Выводы

В рамках данной работы мы показали, что качество линейной регрессии не изменится при умножении признаков на обратимую матрицу.  

С учетом этого вывода мы предложили и реализовали простой алгоритм преобразования данных, который заключается в умножении исходной матрицы признаков на квадратную обратимую матрицу подходящей размерности (с количеством столбцов (строк) равным количеству признаков).

Мы проверили работу алгоритма и подтвердили корректность результатов, построив модели линейной регрессии на исходных и преобразованных данных и оценив коэффициент детерминации на кросс-валидации, а также успешно реализовав обратное преобразование.